In [217]:
import pandas as pd

In [218]:
item_cate=pd.read_csv('item_categories.csv')
items=pd.read_csv('items.csv')
traindf=pd.read_csv('sales_train.csv')
sample=pd.read_csv('sample_submission.csv')
shops=pd.read_csv('shops.csv')
test=pd.read_csv('test.csv')

In [219]:
display(traindf.head())
display(item_cate.head())
display(items.head())
display(shops.head())
display(test.head())

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [220]:
traindf=pd.merge(traindf,items,how='left',on='item_id')
traindf.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56


In [221]:

traindf.sort_values(['shop_id','item_id'])
#cols = ['shop_id','item_id','item_cnt_day','date']
#cols = ['shop_id','item_id','item_cnt_day','date','item_category_id']
cols = ['shop_id','item_cnt_day','date','item_category_id']
#cols = ['shop_id','item_id','item_cnt_day','item_category_id']
df = pd.DataFrame(index=[], columns=cols)

for i in traindf.date_block_num.unique():
    bymonth=traindf[traindf.date_block_num==i].sort_values('date')
    #tmp=bymonth.groupby(['shop_id','item_id']).sum()['item_cnt_day'].reset_index()
    #tmp=bymonth.groupby(['shop_id','item_category_id','item_id']).sum()['item_cnt_day'].reset_index()
    tmp=bymonth.groupby(['shop_id','item_category_id']).sum()['item_cnt_day'].reset_index()
    tmp['date']=i
    #tmp.rename(columns={'item_cnt_month':'item_cnt_day'})
    df=df.append(tmp)

df.rename(columns={'item_cnt_day':'item_cnt_month'},inplace=True)
df.head()


,shop_id,item_cnt_month,date,item_category_id
0,0,53.0,0,2
1,0,28.0,0,3
2,0,16.0,0,4
3,0,28.0,0,5
4,0,65.0,0,6


In [222]:
#df1=pd.DataFrame(df.groupby(['shop_id','item_id']).mean()).reset_index()
#df1=df.drop('date',axis=1)


In [223]:
df

,shop_id,item_cnt_month,date,item_category_id
0,0,53.0,0,2
1,0,28.0,0,3
2,0,16.0,0,4
3,0,28.0,0,5
4,0,65.0,0,6
...,...,...,...,...
1755,59,13.0,33,72
1756,59,2.0,33,73
1757,59,12.0,33,75
1758,59,13.0,33,79


In [224]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
x=df.drop(['item_cnt_month','date'],axis=1)
y=df[['item_cnt_month']]
x_train,x_test,y_train,y_test=train_test_split(x,y)
print(x_train.shape,x_test.shape)
x_train

(48891, 2) (16298, 2)


,shop_id,item_category_id
1570,51,33
1734,53,75
1753,56,35
689,21,75
500,19,20
...,...,...
976,30,2
7,2,21
236,7,16
1433,46,61


In [225]:
rfg=RandomForestRegressor()
rfg.fit(x_train,y_train)
sc_tr=rfg.score(x_train,y_train)
sc_te=rfg.score(x_test,y_test)
print(sc_tr,sc_te)

C:\Users\shunt\anaconda3\envs\ch10\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8110448026149539 0.728501771774317


In [226]:
test=pd.merge(test,items,how='left',on='item_id')


In [227]:
xpredict=test[['shop_id','item_category_id']]


In [230]:
predict1=rfg.predict(xpredict)
test['item_cnt_month']=predict1

In [233]:
test[['ID','item_cnt_month']].to_csv('submission.csv',index=False)